In [1]:
import numpy as np
import os
import pandas as pd
from skimage import io
from skimage import color, exposure, transform
from PIL import Image 
import cv2
import matplotlib
import matplotlib.pyplot as plt

from skimage import data, img_as_float
from skimage import exposure

import keras
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Convolution2D 
from keras.layers import Input, MaxPooling2D, Dense, Dropout, Activation, Flatten
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
from keras.utils import np_utils 

from keras.preprocessing.image import ImageDataGenerator
from sklearn.cross_validation import train_test_split

from tensorflow.contrib import learn
from tensorflow.contrib.learn.python.learn.estimators import model_fn as model_fn_lib

Using TensorFlow backend.
/Users/alexandrosstergiou/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def analyse_images(imgs_paths):
    
    data = []
    i = 0
    for img_path in imgs_paths:
        img_class = int(img_path.split('/')[-2])
        
        
        """
        #If image has width and height that are not equal
        #or overall size is not of 48x48
        
        size = 48,48
        img = Image.open(img_path)
        img = img.thumbnail(size, Image.ANTIALIAS)
        os.remove(img_path)
        img.save(img_path, "JPEG")
        """
        
        img = io.imread(img_path)
        

        # Histogram
        img_h = exposure.equalize_hist(img)
        
        data.append([img_h,img_class])
        
        i = i+1
        
        #Print every 10000 images
        if ((i % 10000)== 0):
            print ("Images processed: ")+str(i)
        
    return data




In [3]:
directory = 'SGTSD/Images/'
paths = []
number_signs = 0

for sub_dir in os.listdir(directory):
    sd = directory+sub_dir+'/'
    if (sub_dir != ".DS_Store"):
        number_signs = number_signs+1
        for files in os.listdir(sd):
            if (files.endswith(".csv")==False):
                paths.append(directory+sub_dir+'/'+files)



np.random.shuffle(paths)
data = analyse_images(paths)
elements = []
classes = []
for d in data:
    elements.append(d[0])
    classes.append(d[1])

    
direct = 'val_set/'
val_paths = []
for sub_dir in os.listdir(direct):
    sd = direct+sub_dir+'/'
    if (sub_dir != ".DS_Store"):
        for files in os.listdir(sd):
            if (files.endswith(".csv")==False) and (files.endswith(".DS_Store")==False):
                val_paths.append(direct+sub_dir+'/'+files)
                
np.random.shuffle(val_paths)
val_data = analyse_images(val_paths)
val_elements = []
val_classes = []
for vd in val_data:
    val_elements.append(vd[0])
    val_classes.append(vd[1])

/Users/alexandrosstergiou/anaconda/lib/python2.7/site-packages/skimage/exposure/exposure.py:63: UserWarning: This might be a color image. The histogram will be computed on the flattened image. You can instead apply this function to each color channel.
  warn("This might be a color image. The histogram will be "


Images processed: 10000
Images processed: 20000
Images processed: 30000
Images processed: 40000
Images processed: 50000
Images processed: 60000
Images processed: 70000
Images processed: 80000
Images processed: 90000
Images processed: 100000
Images processed: 110000
Images processed: 120000
Images processed: 130000
Images processed: 140000
Images processed: 150000
Images processed: 160000
Images processed: 170000
Images processed: 180000
Images processed: 190000
Images processed: 200000


In [4]:
def cnn_model(height,width,depth,number_signs):
    
    
    inp = Input(shape=(height, width, depth))
    
    act= keras.layers.advanced_activations.LeakyReLU(alpha=0.001)
    #act = keras.layers.advanced_activations.ELU(alpha=0.001)
    conv_1 = Convolution2D(32, (3, 3), padding='same', activation=act)(inp)
    conv_2 = Convolution2D(32, (3, 3), padding='same', activation=act)(conv_1)
    
    pool_1 = MaxPooling2D((2, 2), data_format="channels_last")(conv_2)
    drop_1 = Dropout(0.25)(pool_1)

    
    conv_3 = Convolution2D(64, (3, 3), padding='same', activation=act)(drop_1)
    conv_4 = Convolution2D(64, (3, 3), padding='same', activation=act)(conv_3)

    pool_2 = MaxPooling2D((2, 2), data_format="channels_last")(conv_4)
    drop_2 = Dropout(0.25)(pool_2)
    
    conv_5 = Convolution2D(128, (3, 3), padding='same', activation=act)(drop_2)
    conv_6 = Convolution2D(128, (3, 3), padding='same', activation=act)(conv_5)

    pool_3 = MaxPooling2D((2, 2), data_format="channels_last")(conv_6)
    drop_3 = Dropout(0.25)(pool_3)
    
    
    
    flat = Flatten()(drop_3)
    hidden = Dense(256, activation=act)(flat)
    drop_4 = Dropout(0.5)(hidden)
    out = Dense(number_signs, activation='softmax')(drop_4)

    model = Model(inputs=inp, outputs=out)

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    model.summary()
    
    
    return model

In [5]:
model = cnn_model(48,48,3,number_signs)

/Users/alexandrosstergiou/anaconda/lib/python2.7/site-packages/keras/activations.py:89: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  ).format(identifier=identifier.__class__.__name__))


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 48, 48, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 24, 24, 64)        36928     
__________

In [7]:
batch_size = 32
nb_epochs = 30


X_train = np.array(elements, dtype='float32')
Y_train = np.eye(number_signs, dtype='uint8')[classes]

X_val = np.array(val_elements, dtype='float32')
Y_val = np.eye(number_signs, dtype='uint8')[val_classes]

history = model.fit(X_train, Y_train,batch_size=batch_size,epochs=nb_epochs, validation_data=(X_val,Y_val))

score = model.evaluate(X_val, Y_val, verbose=0)

print('Test score:', score[0])
print('Test accuracy:', score[1])

Train on 209415 samples, validate on 2500 samples
Epoch 1/30
209415/209415 [==============================] - 2866s - loss: 0.1799 - acc: 0.9494 - val_loss: 3.0277 - val_acc: 0.5424
Epoch 2/30
  9920/209415 [>.............................] - ETA: 2667s - loss: 0.0108 - acc: 0.9968

KeyboardInterrupt: 

In [ ]:
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.savefig('CNN_1-accuracy.png')
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.savefig('CNN_1-loss.png')

In [ ]:
import sys
orig_stdout = sys.stdout
f = open('Model_Summary.txt', 'w')
sys.stdout = f
print(model.summary())
sys.stdout = orig_stdout
f.close()

In [ ]:
orig_stdout = sys.stdout
f = open('History.txt', 'w')
sys.stdout = f
print(history.history)
print('Test score:', score[0])
print('Test accuracy:', score[1])
sys.stdout = orig_stdout
f.close()